<a href="https://colab.research.google.com/github/jonanew/noise_detection/blob/main/DBSCAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rdflib

# Install noise package for pink noise generation
!pip install noise

from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for noise: filename=noise-1.2.2-cp311-cp311-linux_x86_64.whl size=56278 sha256=4209298aec21c8325fe2a1f06933da245aab07966596065aa19852fb0983794c
  Stored in directory: /root/.cache/pip/wheels/39/25/2e/af6d1bcc91a8f99af0f651f8718b9ab999720a21c6d4149091
Successfully built noise
Mounted at /content/drive


In [2]:
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import DBSCAN
from sklearn.model_selection import KFold
from sklearn.metrics import silhouette_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import warnings
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
from rdflib import Graph
from google.colab import drive

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Paths to sensor reading csv file data and knowledge graph
data_path = "/content/drive/My Drive/ColabNotebooks/motion_sensor_readings.csv"
kg_path = "/content/drive/My Drive/ColabNotebooks/motion_sense_ssn_kg.ttl"

# Load knowledge graph
g = Graph()
try:
    g.parse(kg_path, format='turtle')
except Exception as e:
    print(f"Error parsing knowledge graph: {e}")
    exit(1)

# Query for participant characteristics
query = """
PREFIX ms: <http://example.org/motion-sense#>
SELECT ?p ?code ?age ?gender ?height ?weight
WHERE {
    ?p a ms:Participant ;
       ms:hasCode ?code ;
       ms:hasAge ?age ;
       ms:hasGender ?gender ;
       ms:hasHeightCm ?height ;
       ms:hasWeightKg ?weight .
}
"""
results = g.query(query)
participants = []
for row in results:
    participants.append({
        'participant': str(row.code).strip().upper(),
        'age': int(row.age),
        'gender': int(row.gender),
        'height': float(row.height),
        'weight': float(row.weight)
    })
participants_df = pd.DataFrame(participants)

if participants_df.empty:
    print("Error: No participants found in knowledge graph.")
    exit(1)

# Query for activity plausible ranges
query_ranges = """
PREFIX activity: <http://example.org/activity-recognition#>
SELECT ?activityCode ?min ?max
WHERE {
    ?act a activity:Activity ;
         activity:hasActivityCode ?activityCode ;
         activity:hasMinAcceleration ?min ;
         activity:hasMaxAcceleration ?max .
}
"""
results_ranges = g.query(query_ranges)
activity_ranges = {}
for row in results_ranges:
    activity_code = str(row.activityCode).strip().lower()
    activity_ranges[activity_code] = {
        'min': float(row.min),
        'max': float(row.max)
    }

if not activity_ranges:
    print("Error: No activity ranges found in knowledge graph.")
    exit(1)

# Load sensor readings csv file data
try:
    df = pd.read_csv(data_path)
    print(df)
except Exception as e:
    print(f"Error loading sensor data: {e}")
    exit(1)

# Parse sensor_id with validation
def parse_sensor_id(sid):
    parts = sid.split('_')
    if len(parts) >= 5:
        return parts[0].strip().upper(), parts[3], parts[4]
    return None, None, None

df['participant'], df['activity'], df['sensor_type'] = zip(*df['sensor_id'].apply(parse_sensor_id))
df = df.dropna(subset=['participant'])

# Filter for accelerometer data
df = df[(df['sensor_type'] == 'Accelerometer') &
        (df['property'].isin(['UserAccelerationX', 'UserAccelerationY', 'UserAccelerationZ']))]

# Convert measurement to numeric, drop invalid entries
df['measurement'] = pd.to_numeric(df['measurement'], errors='coerce')
df = df.dropna(subset=['measurement'])

if df.empty:
    print("Error: No valid accelerometer data after cleaning.")
    exit(1)

# Pivot to get X, Y, Z
acc_df = df.pivot_table(index=['timestamp', 'sensor_id', 'participant', 'activity'],
                        columns='property', values='measurement', aggfunc='first').reset_index()

# Drop rows with missing or invalid acceleration components
acc_df = acc_df.dropna(subset=['UserAccelerationX', 'UserAccelerationY', 'UserAccelerationZ'])
for col in ['UserAccelerationX', 'UserAccelerationY', 'UserAccelerationZ']:
    acc_df[col] = pd.to_numeric(acc_df[col], errors='coerce')
acc_df = acc_df.dropna(subset=['UserAccelerationX', 'UserAccelerationY', 'UserAccelerationZ'])

if acc_df.empty:
    print("Error: No valid data after pivot and cleaning.")
    exit(1)

# Compute magnitude
acc_df['magnitude'] = np.sqrt(acc_df['UserAccelerationX']**2 +
                              acc_df['UserAccelerationY']**2 +
                              acc_df['UserAccelerationZ']**2)

# Compute global mean and std for magnitude (for traditional Z-score)
global_magnitude_mean = acc_df['magnitude'].mean()
global_magnitude_std = acc_df['magnitude'].std()
print(f"Global magnitude stats: Mean = {global_magnitude_mean:.6f}, Std = {global_magnitude_std:.6f}")

# Compute per-activity stats for reference on cleaned data
activity_stats_df = acc_df.groupby('activity')['magnitude'].agg(['mean', 'std', 'count']).reset_index()
activity_stats_df.columns = ['activity', 'actual_mean', 'actual_std', 'sample_count']

# Ensure sufficient samples and smooth std
activity_stats_df = activity_stats_df[activity_stats_df['sample_count'] >= 100]
global_activity_std = activity_stats_df['actual_std'].mean()
activity_stats_df['actual_std'] = activity_stats_df['actual_std'].apply(lambda x: max(x, global_activity_std * 0.1))

# Debug: Print per-activity stats
print("Per-activity magnitude stats:\n", activity_stats_df[['activity', 'actual_mean', 'actual_std', 'sample_count']])

# Merge with participant characteristics for clustering
merged_df = pd.merge(acc_df, participants_df, on='participant', how='inner')

if merged_df.empty:
    print("Error: No matching participants found after merging.")
    exit(1)

# Subsample the data to reduce memory usage
subsample_fraction = 0.2  # Use 10% of the data
subsampled_df = merged_df.sample(frac=subsample_fraction, random_state=42)
print(f"Subsampled data size: {len(subsampled_df)}")

# Initialize 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
results_list = []

# Iterate over folds
for fold, (train_index, test_index) in enumerate(kf.split(subsampled_df), 1):
    print(f"Processing Fold {fold}")
    train_df = subsampled_df.iloc[train_index].copy()
    test_df = subsampled_df.iloc[test_index].copy()
    print(f"Training set size: {len(train_df)}, Test set size: {len(test_df)}")

    # Prepare data for clustering on training set, including magnitude
    X_cluster_train = train_df[['age', 'gender', 'height', 'weight', 'magnitude', 'activity']]
    scaler = RobustScaler()
    X_numeric_train = scaler.fit_transform(X_cluster_train[['age', 'gender', 'height', 'weight', 'magnitude']])
    X_activity_train = pd.get_dummies(X_cluster_train['activity'], prefix='activity')
    X_scaled_train = np.hstack([X_numeric_train, X_activity_train])

    # Prepare data for test set
    X_cluster_test = test_df[['age', 'gender', 'height', 'weight', 'magnitude', 'activity']]
    X_numeric_test = scaler.transform(X_cluster_test[['age', 'gender', 'height', 'weight', 'magnitude']])
    X_activity_test = pd.get_dummies(X_cluster_test['activity'], prefix='activity')
    X_activity_test = X_activity_test.reindex(columns=X_activity_train.columns, fill_value=0)
    X_scaled_test = np.hstack([X_numeric_test, X_activity_test])

    # Generate k-distance plot to determine initial eps
    min_samples = 50  # Default min_samples
    neighbors = NearestNeighbors(n_neighbors=min_samples).fit(X_scaled_train)
    distances, indices = neighbors.kneighbors(X_scaled_train)
    k_distances = np.sort(distances[:, min_samples - 1])
    plt.figure(figsize=(10, 6))
    plt.plot(k_distances)
    plt.title(f'K-Distance Plot (Distance to {min_samples}th Nearest Neighbor) - Fold {fold}')
    plt.xlabel('Points Sorted by Distance')
    plt.ylabel('Distance')
    plt.grid(True)
    plt.savefig(f'/content/drive/My Drive/ColabNotebooks/k_distance_plot_fold_{fold}.png')
    plt.close()

    # Suggest eps based on the knee point (e.g., 95th percentile of k-distances)
    suggested_eps = np.percentile(k_distances, 95)
    print(f"Suggested eps based on 95th percentile of k-distances for Fold {fold}: {suggested_eps:.3f}")

    # Parameter tuning using silhouette score
    eps_values = np.arange(0.06, 0.08, 0.01)  # Adjusted range based on knee
    min_samples_values = [20, 30, 50, 60, 70]  # Adjusted range
    best_score = -1
    best_params = {'eps': 0, 'min_samples': 0}

    for eps in eps_values:
        for min_samples in min_samples_values:
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            labels = dbscan.fit_predict(X_scaled_train)
            if len(np.unique(labels)) > 1 and -1 in labels:
                score = silhouette_score(X_scaled_train, labels, sample_size=5000)
                if score > best_score:
                    best_score = score
                    best_params = {'eps': eps, 'min_samples': min_samples}

    print(f"Best parameters for Fold {fold}: eps={best_params['eps']:.3f}, min_samples={best_params['min_samples']}, Best Silhouette Score={best_score:.3f}")

    # Use best parameters for clustering
    dbscan = DBSCAN(eps=best_params['eps'], min_samples=best_params['min_samples'])
    train_cluster_labels = dbscan.fit_predict(X_scaled_train)
    test_cluster_labels = dbscan.fit_predict(X_scaled_test)

    # Add cluster labels to train and test dataframes
    train_df['cluster'] = train_cluster_labels
    test_df['cluster'] = test_cluster_labels

    # Combine train and test dataframes
    acc_df = pd.concat([train_df, test_df], axis=0)

    # Assign cluster mean and std for Z-score calculation using DBSCAN clusters
    cluster_means = train_df[train_df['cluster'] != -1].groupby('cluster')['magnitude'].mean().to_dict()
    cluster_stds = train_df[train_df['cluster'] != -1].groupby('cluster')['magnitude'].std().to_dict()
    acc_df['cluster_mu'] = acc_df['cluster'].map(cluster_means)
    acc_df['cluster_sigma'] = acc_df['cluster'].map(cluster_stds).fillna(global_activity_std)
    acc_df.loc[acc_df['cluster'] == -1, 'cluster_mu'] = global_magnitude_mean
    acc_df.loc[acc_df['cluster'] == -1, 'cluster_sigma'] = global_magnitude_std

    # Compute Z-scores using cluster-based statistics
    acc_df['Z_score'] = (acc_df['magnitude'] - acc_df['cluster_mu']) / acc_df['cluster_sigma'].replace(0, np.nan)
    acc_df['traditional_Z_score'] = (acc_df['magnitude'] - global_magnitude_mean) / global_magnitude_std

    # Add global mean and std to output
    acc_df['global_magnitude_mean'] = global_magnitude_mean
    acc_df['global_magnitude_std'] = global_magnitude_std

    # Apply activity-specific plausible ranges
    acc_df['activity'] = acc_df['activity'].str.lower()
    default_min_plausible = 0
    default_max_plausible = 10
    acc_df['min_plausible'] = acc_df['activity'].map(lambda x: activity_ranges.get(x, {'min': default_min_plausible})['min'])
    acc_df['max_plausible'] = acc_df['activity'].map(lambda x: activity_ranges.get(x, {'max': default_max_plausible})['max'])

    # Determine if observations are outside plausible range
    acc_df['outside_plausible'] = (acc_df['magnitude'] < acc_df['min_plausible']) | (acc_df['magnitude'] > acc_df['max_plausible'])

    # Determine anomaly
    threshold = 2
    acc_df['is_anomaly'] = np.abs(acc_df['Z_score']) > threshold
    acc_df['is_anomaly_traditional'] = np.abs(acc_df['traditional_Z_score']) > threshold

    # Label noise
    acc_df['is_noise'] = acc_df['is_anomaly'] & acc_df['outside_plausible']
    acc_df['is_noise_traditional'] = acc_df['is_anomaly_traditional'] & acc_df['outside_plausible']

    # Round numerical columns in acc_df to 2 decimal places
    numerical_cols = acc_df.select_dtypes(include=['float64', 'int64']).columns
    acc_df[numerical_cols] = acc_df[numerical_cols].round(2)

    # Function to introduce Gaussian noise
    def introduce_noise(df, noise_level=0.1, noise_fraction=0.05):
        noisy_df = df.copy()
        mask = np.random.rand(len(noisy_df)) < noise_fraction
        noise = np.random.normal(0, noise_level, size=len(noisy_df))
        noisy_df['magnitude'] = noisy_df['magnitude'] + noise * mask
        noisy_df['is_true_noise'] = mask.astype(int)  # Ground truth: 1 for noisy, 0 for normal
        return noisy_df

    # List of noise fractions to test
    noise_fractions = [0.02, 0.05, 0.10, 0.20]
    for noise_fraction in noise_fractions:
        print(f"Processing noise fraction: {noise_fraction*100}% in Fold {fold}")
        # Introduce noise to the dataset
        noisy_df = introduce_noise(acc_df, noise_level=2 * global_magnitude_std, noise_fraction=noise_fraction)

        # Apply anomaly detection - Hybrid method
        noisy_df['Z_score'] = (noisy_df['magnitude'] - noisy_df['cluster_mu']) / noisy_df['cluster_sigma'].replace(0, np.nan)
        noisy_df['is_anomaly'] = np.abs(noisy_df['Z_score']) > threshold
        noisy_df['outside_plausible'] = (noisy_df['magnitude'] < noisy_df['min_plausible']) | (noisy_df['magnitude'] > noisy_df['max_plausible'])
        noisy_df['is_noise'] = noisy_df['is_anomaly'] & noisy_df['outside_plausible']

        # Apply anomaly detection - Traditional method
        noisy_df['traditional_Z_score'] = (noisy_df['magnitude'] - global_magnitude_mean) / global_magnitude_std
        noisy_df['is_anomaly_traditional'] = np.abs(noisy_df['traditional_Z_score']) > threshold
        noisy_df['is_noise_traditional'] = noisy_df['is_anomaly_traditional'] & noisy_df['outside_plausible']

        # Evaluate performance
        y_true = noisy_df['is_true_noise']
        y_pred_hybrid = noisy_df['is_noise'].astype(int)
        y_pred_traditional = noisy_df['is_noise_traditional'].astype(int)

        precision_hybrid = precision_score(y_true, y_pred_hybrid)
        recall_hybrid = recall_score(y_true, y_pred_hybrid)
        f1_hybrid = f1_score(y_true, y_pred_hybrid)

        precision_traditional = precision_score(y_true, y_pred_traditional)
        recall_traditional = recall_score(y_true, y_pred_traditional)
        f1_traditional = f1_score(y_true, y_pred_traditional)

        # Calculate percentage of noise detected
        total_noise_points = y_true.sum()
        hybrid_detected = y_pred_hybrid[y_true == 1].sum()
        traditional_detected = y_pred_traditional[y_true == 1].sum()
        hybrid_noise_percentage = (hybrid_detected / total_noise_points * 100) if total_noise_points > 0 else 0
        traditional_noise_percentage = (traditional_detected / total_noise_points * 100) if total_noise_points > 0 else 0

        # Store results for this noise level
        results_list.append({
            'Algorithm': 'DBSCAN',
            'Fold': fold,
            'Noise Fraction (%)': noise_fraction * 100,
            'Method': 'Hybrid (Cluster-based)',
            'Precision': precision_hybrid,
            'Recall': recall_hybrid,
            'F1 Score': f1_hybrid,
            'Noise Detected (%)': hybrid_noise_percentage
        })
        results_list.append({
            'Algorithm': 'DBSCAN',
            'Fold': fold,
            'Noise Fraction (%)': noise_fraction * 100,
            'Method': 'Traditional (Global)',
            'Precision': precision_traditional,
            'Recall': recall_traditional,
            'F1 Score': f1_traditional,
            'Noise Detected (%)': traditional_noise_percentage
        })

# Create consolidated results table
results_df = pd.DataFrame(results_list)
print("\nConsolidated Evaluation Results:")
print(results_df.groupby(['Noise Fraction (%)', 'Method']).agg({
    'Precision': 'mean',
    'Recall': 'mean',
    'F1 Score': 'mean',
    'Noise Detected (%)': 'mean'
}).reset_index())

# Save all results to first CSV
all_results_path = '/content/drive/My Drive/ColabNotebooks/all_results_clustered_optimized_dbscan.csv'
try:
    acc_df.to_csv(all_results_path, index=False)
    print(f"All results saved to '{all_results_path}'.")
except Exception as e:
    print(f"Error saving all results: {e}")

# Save noisy dataset and results for the last fold
try:
    noisy_df.to_csv('/content/drive/My Drive/ColabNotebooks/noisy_results_dbscan.csv', index=False)
    print(f"Noisy dataset saved to '/content/drive/My Drive/ColabNotebooks/noisy_results_dbscan.csv'.")
    results_df.to_csv('/content/drive/My Drive/ColabNotebooks/evaluation_results_dbscan.csv', index=False)
    print(f"Evaluation results saved to '/content/drive/My Drive/ColabNotebooks/evaluation_results_dbscan.csv'.")
except Exception as e:
    print(f"Error saving files: {e}")

# Debug: Print summary for the last noise level of the last fold
print("\nZ-score stats (noisy dataset, last noise level of last fold):\n", noisy_df[['Z_score', 'traditional_Z_score']].describe())
print("Noise detection summary (hybrid, noisy dataset):\n", noisy_df['is_noise'].value_counts())
print("Noise detection summary (traditional, noisy dataset):\n", noisy_df['is_noise_traditional'].value_counts())
print(f"Total noise points introduced (last noise level of last fold): {total_noise_points}")
print(f"Hybrid method detected: {hybrid_detected} ({hybrid_noise_percentage:.2f}%)")
print(f"Traditional method detected: {traditional_detected} ({traditional_noise_percentage:.2f}%)")

                        timestamp                        sensor_id  \
0        2017-10-01T12:00:00.000Z       P001_Trial_1_dws_Gyroscope   
1        2017-10-01T12:00:00.000Z       P001_Trial_1_dws_Gyroscope   
2        2017-10-01T12:00:00.000Z       P001_Trial_1_dws_Gyroscope   
3        2017-10-01T12:00:00.000Z   P001_Trial_1_dws_Accelerometer   
4        2017-10-01T12:00:00.000Z   P001_Trial_1_dws_Accelerometer   
...                           ...                              ...   
4605955  2017-10-01T16:15:53.180Z      P024_Trial_16_jog_Gyroscope   
4605956  2017-10-01T16:15:53.180Z      P024_Trial_16_jog_Gyroscope   
4605957  2017-10-01T16:15:53.180Z  P024_Trial_16_jog_Accelerometer   
4605958  2017-10-01T16:15:53.180Z  P024_Trial_16_jog_Accelerometer   
4605959  2017-10-01T16:15:53.180Z  P024_Trial_16_jog_Accelerometer   

         measurement           property  
0           1.528132       AttitudeRoll  
1          -0.733896      AttitudePitch  
2           0.696372        Attit

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import DBSCAN
from sklearn.model_selection import KFold
from sklearn.metrics import silhouette_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import warnings
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
from scipy.stats import gennorm
from rdflib import Graph
from google.colab import drive

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Paths to sensor reading csv file data and knowledge graph
data_path = "/content/drive/My Drive/ColabNotebooks/motion_sensor_readings.csv"
kg_path = "/content/drive/My Drive/ColabNotebooks/motion_sense_ssn_kg.ttl"

# Load knowledge graph
g = Graph()
try:
    g.parse(kg_path, format='turtle')
except Exception as e:
    print(f"Error parsing knowledge graph: {e}")
    exit(1)

# Query for participant characteristics
query = """
PREFIX ms: <http://example.org/motion-sense#>
SELECT ?p ?code ?age ?gender ?height ?weight
WHERE {
    ?p a ms:Participant ;
       ms:hasCode ?code ;
       ms:hasAge ?age ;
       ms:hasGender ?gender ;
       ms:hasHeightCm ?height ;
       ms:hasWeightKg ?weight .
}
"""
results = g.query(query)
participants = []
for row in results:
    participants.append({
        'participant': str(row.code).strip().upper(),
        'age': int(row.age),
        'gender': int(row.gender),
        'height': float(row.height),
        'weight': float(row.weight)
    })
participants_df = pd.DataFrame(participants)

if participants_df.empty:
    print("Error: No participants found in knowledge graph.")
    exit(1)

# Query for activity plausible ranges
query_ranges = """
PREFIX activity: <http://example.org/activity-recognition#>
SELECT ?activityCode ?min ?max
WHERE {
    ?act a activity:Activity ;
         activity:hasActivityCode ?activityCode ;
         activity:hasMinAcceleration ?min ;
         activity:hasMaxAcceleration ?max .
}
"""
results_ranges = g.query(query_ranges)
activity_ranges = {}
for row in results_ranges:
    activity_code = str(row.activityCode).strip().lower()
    activity_ranges[activity_code] = {
        'min': float(row.min),
        'max': float(row.max)
    }

if not activity_ranges:
    print("Error: No activity ranges found in knowledge graph.")
    exit(1)

# Load sensor readings csv file data
try:
    df = pd.read_csv(data_path)
    print(df)
except Exception as e:
    print(f"Error loading sensor data: {e}")
    exit(1)

# Parse sensor_id with validation
def parse_sensor_id(sid):
    parts = sid.split('_')
    if len(parts) >= 5:
        return parts[0].strip().upper(), parts[3], parts[4]
    return None, None, None

df['participant'], df['activity'], df['sensor_type'] = zip(*df['sensor_id'].apply(parse_sensor_id))
df = df.dropna(subset=['participant'])

# Filter for accelerometer data
df = df[(df['sensor_type'] == 'Accelerometer') &
        (df['property'].isin(['UserAccelerationX', 'UserAccelerationY', 'UserAccelerationZ']))]

# Convert measurement to numeric, drop invalid entries
df['measurement'] = pd.to_numeric(df['measurement'], errors='coerce')
df = df.dropna(subset=['measurement'])

if df.empty:
    print("Error: No valid accelerometer data after cleaning.")
    exit(1)

# Pivot to get X, Y, Z
acc_df = df.pivot_table(index=['timestamp', 'sensor_id', 'participant', 'activity'],
                        columns='property', values='measurement', aggfunc='first').reset_index()

# Drop rows with missing or invalid acceleration components
acc_df = acc_df.dropna(subset=['UserAccelerationX', 'UserAccelerationY', 'UserAccelerationZ'])
for col in ['UserAccelerationX', 'UserAccelerationY', 'UserAccelerationZ']:
    acc_df[col] = pd.to_numeric(acc_df[col], errors='coerce')
acc_df = acc_df.dropna(subset=['UserAccelerationX', 'UserAccelerationY', 'UserAccelerationZ'])

if acc_df.empty:
    print("Error: No valid data after pivot and cleaning.")
    exit(1)

# Compute magnitude
acc_df['magnitude'] = np.sqrt(acc_df['UserAccelerationX']**2 +
                              acc_df['UserAccelerationY']**2 +
                              acc_df['UserAccelerationZ']**2)

# Compute global mean and std for magnitude (for traditional Z-score)
global_magnitude_mean = acc_df['magnitude'].mean()
global_magnitude_std = acc_df['magnitude'].std()
print(f"Global magnitude stats: Mean = {global_magnitude_mean:.6f}, Std = {global_magnitude_std:.6f}")

# Compute per-activity stats for reference on cleaned data
activity_stats_df = acc_df.groupby('activity')['magnitude'].agg(['mean', 'std', 'count']).reset_index()
activity_stats_df.columns = ['activity', 'actual_mean', 'actual_std', 'sample_count']

# Ensure sufficient samples and smooth std
activity_stats_df = activity_stats_df[activity_stats_df['sample_count'] >= 100]
global_activity_std = activity_stats_df['actual_std'].mean()
activity_stats_df['actual_std'] = activity_stats_df['actual_std'].apply(lambda x: max(x, global_activity_std * 0.1))

# Debug: Print per-activity stats
print("Per-activity magnitude stats:\n", activity_stats_df[['activity', 'actual_mean', 'actual_std', 'sample_count']])

# Merge with participant characteristics for clustering
merged_df = pd.merge(acc_df, participants_df, on='participant', how='inner')

if merged_df.empty:
    print("Error: No matching participants found after merging.")
    exit(1)

# Subsample the data to reduce memory usage
subsample_fraction = 0.4  # Use 40% of the data
subsampled_df = merged_df.sample(frac=subsample_fraction, random_state=42)
print(f"Subsampled data size: {len(subsampled_df)}")

# Initialize 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
results_list = []

# Iterate over folds
for fold, (train_index, test_index) in enumerate(kf.split(subsampled_df), 1):
    print(f"Processing Fold {fold}")
    train_df = subsampled_df.iloc[train_index].copy()
    test_df = subsampled_df.iloc[test_index].copy()
    print(f"Training set size: {len(train_df)}, Test set size: {len(test_df)}")

    # Prepare data for clustering on training set, including magnitude
    X_cluster_train = train_df[['age', 'gender', 'height', 'weight', 'magnitude', 'activity']]
    scaler = RobustScaler()
    X_numeric_train = scaler.fit_transform(X_cluster_train[['age', 'gender', 'height', 'weight', 'magnitude']])
    X_activity_train = pd.get_dummies(X_cluster_train['activity'], prefix='activity')
    X_scaled_train = np.hstack([X_numeric_train, X_activity_train])

    # Prepare data for test set
    X_cluster_test = test_df[['age', 'gender', 'height', 'weight', 'magnitude', 'activity']]
    X_numeric_test = scaler.transform(X_cluster_test[['age', 'gender', 'height', 'weight', 'magnitude']])
    X_activity_test = pd.get_dummies(X_cluster_test['activity'], prefix='activity')
    X_activity_test = X_activity_test.reindex(columns=X_activity_train.columns, fill_value=0)
    X_scaled_test = np.hstack([X_numeric_test, X_activity_test])

    # Generate k-distance plot to determine initial eps
    min_samples = 50  # Default min_samples
    neighbors = NearestNeighbors(n_neighbors=min_samples).fit(X_scaled_train)
    distances, indices = neighbors.kneighbors(X_scaled_train)
    k_distances = np.sort(distances[:, min_samples - 1])
    plt.figure(figsize=(10, 6))
    plt.plot(k_distances)
    plt.title(f'K-Distance Plot (Distance to {min_samples}th Nearest Neighbor) - Fold {fold}')
    plt.xlabel('Points Sorted by Distance')
    plt.ylabel('Distance')
    plt.grid(True)
    plt.savefig(f'/content/drive/My Drive/ColabNotebooks/k_distance_plot_fold_{fold}.png')
    plt.close()

    # Suggest eps based on the knee point (e.g., 95th percentile of k-distances)
    suggested_eps = np.percentile(k_distances, 95)
    print(f"Suggested eps based on 95th percentile of k-distances for Fold {fold}: {suggested_eps:.3f}")

    # Parameter tuning using silhouette score
    eps_values = np.arange(0.1, 1.0, 0.1)  # Wider range to capture more noise
    min_samples_values = [10, 20, 30, 40, 50]  # Adjusted range
    best_score = -1
    best_params = {'eps': 0, 'min_samples': 0}

    for eps in eps_values:
        for min_samples in min_samples_values:
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            labels = dbscan.fit_predict(X_scaled_train)
            if len(np.unique(labels)) > 1 and -1 in labels:
                score = silhouette_score(X_scaled_train, labels, sample_size=5000)
                if score > best_score:
                    best_score = score
                    best_params = {'eps': eps, 'min_samples': min_samples}

    print(f"Best parameters for Fold {fold}: eps={best_params['eps']:.1f}, min_samples={best_params['min_samples']}, Best Silhouette Score={best_score:.3f}")

    # Use best parameters for clustering
    dbscan = DBSCAN(eps=best_params['eps'], min_samples=best_params['min_samples'])
    train_cluster_labels = dbscan.fit_predict(X_scaled_train)
    test_cluster_labels = dbscan.fit_predict(X_scaled_test)

    # Add cluster labels to train and test dataframes
    train_df['cluster'] = train_cluster_labels
    test_df['cluster'] = test_cluster_labels

    # Combine train and test dataframes
    acc_df = pd.concat([train_df, test_df], axis=0)

    # Assign cluster mean and std for Z-score calculation using DBSCAN clusters
    cluster_means = train_df[train_df['cluster'] != -1].groupby('cluster')['magnitude'].mean().to_dict()
    cluster_stds = train_df[train_df['cluster'] != -1].groupby('cluster')['magnitude'].std().to_dict()
    acc_df['cluster_mu'] = acc_df['cluster'].map(cluster_means)
    acc_df['cluster_sigma'] = acc_df['cluster'].map(cluster_stds).fillna(global_activity_std)
    acc_df.loc[acc_df['cluster'] == -1, 'cluster_mu'] = global_magnitude_mean
    acc_df.loc[acc_df['cluster'] == -1, 'cluster_sigma'] = global_magnitude_std

    # Compute Z-scores using cluster-based statistics
    acc_df['Z_score'] = (acc_df['magnitude'] - acc_df['cluster_mu']) / acc_df['cluster_sigma'].replace(0, np.nan)
    acc_df['traditional_Z_score'] = (acc_df['magnitude'] - global_magnitude_mean) / global_magnitude_std

    # Add global mean and std to output
    acc_df['global_magnitude_mean'] = global_magnitude_mean
    acc_df['global_magnitude_std'] = global_magnitude_std

    # Apply activity-specific plausible ranges
    acc_df['activity'] = acc_df['activity'].str.lower()
    default_min_plausible = 0
    default_max_plausible = 10
    acc_df['min_plausible'] = acc_df['activity'].map(lambda x: activity_ranges.get(x, {'min': default_min_plausible})['min'])
    acc_df['max_plausible'] = acc_df['activity'].map(lambda x: activity_ranges.get(x, {'max': default_max_plausible})['max'])

    # Determine if observations are outside plausible range
    acc_df['outside_plausible'] = (acc_df['magnitude'] < acc_df['min_plausible']) | (acc_df['magnitude'] > acc_df['max_plausible'])

    # Determine anomaly
    threshold = 2
    acc_df['is_anomaly'] = np.abs(acc_df['Z_score']) > threshold
    acc_df['is_anomaly_traditional'] = np.abs(acc_df['traditional_Z_score']) > threshold

    # Label noise
    acc_df['is_noise'] = acc_df['is_anomaly'] & acc_df['outside_plausible']
    acc_df['is_noise_traditional'] = acc_df['is_anomaly_traditional'] & acc_df['outside_plausible']

    # Round numerical columns in acc_df to 2 decimal places
    numerical_cols = acc_df.select_dtypes(include=['float64', 'int64']).columns
    acc_df[numerical_cols] = acc_df[numerical_cols].round(2)

    # Function to introduce Generalized Normal noise
    def introduce_noise(df, noise_level=0.1, noise_fraction=0.05):
        noisy_df = df.copy()
        mask = np.random.rand(len(noisy_df)) < noise_fraction
        noise = gennorm.rvs(beta=1.5, loc=0, scale=noise_level, size=len(noisy_df))
        noisy_df['magnitude'] = noisy_df['magnitude'] + noise * mask
        noisy_df['is_true_noise'] = mask.astype(int)  # Ground truth: 1 for noisy, 0 for normal
        return noisy_df

    # List of noise fractions to test
    noise_fractions = [0.02, 0.05, 0.10, 0.20]
    for noise_fraction in noise_fractions:
        print(f"Processing noise fraction: {noise_fraction*100}% in Fold {fold}")
        # Introduce noise to the dataset
        noisy_df = introduce_noise(acc_df, noise_level=2 * global_magnitude_std, noise_fraction=noise_fraction)

        # Apply anomaly detection - Hybrid method
        noisy_df['Z_score'] = (noisy_df['magnitude'] - noisy_df['cluster_mu']) / noisy_df['cluster_sigma'].replace(0, np.nan)
        noisy_df['is_anomaly'] = np.abs(noisy_df['Z_score']) > threshold
        noisy_df['outside_plausible'] = (noisy_df['magnitude'] < noisy_df['min_plausible']) | (noisy_df['magnitude'] > noisy_df['max_plausible'])
        noisy_df['is_noise'] = noisy_df['is_anomaly'] & noisy_df['outside_plausible']

        # Apply anomaly detection - Traditional method
        noisy_df['traditional_Z_score'] = (noisy_df['magnitude'] - global_magnitude_mean) / global_magnitude_std
        noisy_df['is_anomaly_traditional'] = np.abs(noisy_df['traditional_Z_score']) > threshold
        noisy_df['is_noise_traditional'] = noisy_df['is_anomaly_traditional'] & noisy_df['outside_plausible']

        # Evaluate performance
        y_true = noisy_df['is_true_noise']
        y_pred_hybrid = noisy_df['is_noise'].astype(int)
        y_pred_traditional = noisy_df['is_noise_traditional'].astype(int)

        precision_hybrid = precision_score(y_true, y_pred_hybrid)
        recall_hybrid = recall_score(y_true, y_pred_hybrid)
        f1_hybrid = f1_score(y_true, y_pred_hybrid)

        precision_traditional = precision_score(y_true, y_pred_traditional)
        recall_traditional = recall_score(y_true, y_pred_traditional)
        f1_traditional = f1_score(y_true, y_pred_traditional)

        # Calculate percentage of noise detected
        total_noise_points = y_true.sum()
        hybrid_detected = y_pred_hybrid[y_true == 1].sum()
        traditional_detected = y_pred_traditional[y_true == 1].sum()
        hybrid_noise_percentage = (hybrid_detected / total_noise_points * 100) if total_noise_points > 0 else 0
        traditional_noise_percentage = (traditional_detected / total_noise_points * 100) if total_noise_points > 0 else 0

        # Store results for this noise level
        results_list.append({
            'Algorithm': 'DBSCAN',
            'Fold': fold,
            'Noise Fraction (%)': noise_fraction * 100,
            'Method': 'Hybrid (Cluster-based)',
            'Precision': precision_hybrid,
            'Recall': recall_hybrid,
            'F1 Score': f1_hybrid,
            'Noise Detected (%)': hybrid_noise_percentage
        })
        results_list.append({
            'Algorithm': 'DBSCAN',
            'Fold': fold,
            'Noise Fraction (%)': noise_fraction * 100,
            'Method': 'Traditional (Global)',
            'Precision': precision_traditional,
            'Recall': recall_traditional,
            'F1 Score': f1_traditional,
            'Noise Detected (%)': traditional_noise_percentage
        })

# Create consolidated results table
results_df = pd.DataFrame(results_list)
print("\nConsolidated Evaluation Results:")
print(results_df.groupby(['Noise Fraction (%)', 'Method']).agg({
    'Precision': 'mean',
    'Recall': 'mean',
    'F1 Score': 'mean',
    'Noise Detected (%)': 'mean'
}).reset_index())

# Save all results to first CSV
all_results_path = '/content/drive/My Drive/ColabNotebooks/all_results_clustered_optimized_dbscan_gn.csv'
try:
    acc_df.to_csv(all_results_path, index=False)
    print(f"All results saved to '{all_results_path}'.")
except Exception as e:
    print(f"Error saving all results: {e}")

# Save noisy dataset and results for the last fold
try:
    noisy_df.to_csv('/content/drive/My Drive/ColabNotebooks/noisy_results_dbscan_gn.csv', index=False)
    print(f"Noisy dataset saved to '/content/drive/My Drive/ColabNotebooks/noisy_results_dbscan_gn.csv'.")
    results_df.to_csv('/content/drive/My Drive/ColabNotebooks/evaluation_results_dbscan_gn.csv', index=False)
    print(f"Evaluation results saved to '/content/drive/My Drive/ColabNotebooks/evaluation_results_dbscan_gn.csv'.")
except Exception as e:
    print(f"Error saving files: {e}")

# Debug: Print summary for the last noise level of the last fold
print("\nZ-score stats (noisy dataset, last noise level of last fold):\n", noisy_df[['Z_score', 'traditional_Z_score']].describe())
print("Noise detection summary (hybrid, noisy dataset):\n", noisy_df['is_noise'].value_counts())
print("Noise detection summary (traditional, noisy dataset):\n", noisy_df['is_noise_traditional'].value_counts())
print(f"Total noise points introduced (last noise level of last fold): {total_noise_points}")
print(f"Hybrid method detected: {hybrid_detected} ({hybrid_noise_percentage:.2f}%)")
print(f"Traditional method detected: {traditional_detected} ({traditional_noise_percentage:.2f}%)")

                        timestamp                        sensor_id  \
0        2017-10-01T12:00:00.000Z       P001_Trial_1_dws_Gyroscope   
1        2017-10-01T12:00:00.000Z       P001_Trial_1_dws_Gyroscope   
2        2017-10-01T12:00:00.000Z       P001_Trial_1_dws_Gyroscope   
3        2017-10-01T12:00:00.000Z   P001_Trial_1_dws_Accelerometer   
4        2017-10-01T12:00:00.000Z   P001_Trial_1_dws_Accelerometer   
...                           ...                              ...   
4605955  2017-10-01T16:15:53.180Z      P024_Trial_16_jog_Gyroscope   
4605956  2017-10-01T16:15:53.180Z      P024_Trial_16_jog_Gyroscope   
4605957  2017-10-01T16:15:53.180Z  P024_Trial_16_jog_Accelerometer   
4605958  2017-10-01T16:15:53.180Z  P024_Trial_16_jog_Accelerometer   
4605959  2017-10-01T16:15:53.180Z  P024_Trial_16_jog_Accelerometer   

         measurement           property  
0           1.528132       AttitudeRoll  
1          -0.733896      AttitudePitch  
2           0.696372        Attit

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import warnings
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
from scipy.stats import gennorm
from rdflib import Graph


# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Paths to sensor reading csv file data and knowledge graph
data_path = "/content/drive/My Drive/ColabNotebooks/motion_sensor_readings.csv"
kg_path = "/content/drive/My Drive/ColabNotebooks/motion_sense_ssn_kg.ttl"

# Load knowledge graph
g = Graph()
try:
    g.parse(kg_path, format='turtle')
except Exception as e:
    print(f"Error parsing knowledge graph: {e}")
    exit(1)

# Query for participant characteristics
query = """
PREFIX ms: <http://example.org/motion-sense#>
SELECT ?p ?code ?age ?gender ?height ?weight
WHERE {
    ?p a ms:Participant ;
       ms:hasCode ?code ;
       ms:hasAge ?age ;
       ms:hasGender ?gender ;
       ms:hasHeightCm ?height ;
       ms:hasWeightKg ?weight .
}
"""
results = g.query(query)
participants = []
for row in results:
    participants.append({
        'participant': str(row.code).strip().upper(),
        'age': int(row.age),
        'gender': int(row.gender),
        'height': float(row.height),
        'weight': float(row.weight)
    })
participants_df = pd.DataFrame(participants)

if participants_df.empty:
    print("Error: No participants found in knowledge graph.")
    exit(1)

# Query for activity plausible ranges
query_ranges = """
PREFIX activity: <http://example.org/activity-recognition#>
SELECT ?activityCode ?min ?max
WHERE {
    ?act a activity:Activity ;
         activity:hasActivityCode ?activityCode ;
         activity:hasMinAcceleration ?min ;
         activity:hasMaxAcceleration ?max .
}
"""
results_ranges = g.query(query_ranges)
activity_ranges = {}
for row in results_ranges:
    activity_code = str(row.activityCode).strip().lower()
    activity_ranges[activity_code] = {
        'min': float(row.min),
        'max': float(row.max)
    }

if not activity_ranges:
    print("Error: No activity ranges found in knowledge graph.")
    exit(1)

# Load sensor readings csv file data
try:
    df = pd.read_csv(data_path)
    print(df)
except Exception as e:
    print(f"Error loading sensor data: {e}")
    exit(1)

# Parse sensor_id with validation
def parse_sensor_id(sid):
    parts = sid.split('_')
    if len(parts) >= 5:
        return parts[0].strip().upper(), parts[3], parts[4]
    return None, None, None

df['participant'], df['activity'], df['sensor_type'] = zip(*df['sensor_id'].apply(parse_sensor_id))
df = df.dropna(subset=['participant'])

# Filter for accelerometer data
df = df[(df['sensor_type'] == 'Accelerometer') &
        (df['property'].isin(['UserAccelerationX', 'UserAccelerationY', 'UserAccelerationZ']))]

# Convert measurement to numeric, drop invalid entries
df['measurement'] = pd.to_numeric(df['measurement'], errors='coerce')
df = df.dropna(subset=['measurement'])

if df.empty:
    print("Error: No valid accelerometer data after cleaning.")
    exit(1)

# Pivot to get X, Y, Z
acc_df = df.pivot_table(index=['timestamp', 'sensor_id', 'participant', 'activity'],
                        columns='property', values='measurement', aggfunc='first').reset_index()

# Drop rows with missing or invalid acceleration components
acc_df = acc_df.dropna(subset=['UserAccelerationX', 'UserAccelerationY', 'UserAccelerationZ'])
for col in ['UserAccelerationX', 'UserAccelerationY', 'UserAccelerationZ']:
    acc_df[col] = pd.to_numeric(acc_df[col], errors='coerce')
acc_df = acc_df.dropna(subset=['UserAccelerationX', 'UserAccelerationY', 'UserAccelerationZ'])

if acc_df.empty:
    print("Error: No valid data after pivot and cleaning.")
    exit(1)

# Compute magnitude
acc_df['magnitude'] = np.sqrt(acc_df['UserAccelerationX']**2 +
                              acc_df['UserAccelerationY']**2 +
                              acc_df['UserAccelerationZ']**2)

# Compute global mean and std for magnitude (for traditional Z-score)
global_magnitude_mean = acc_df['magnitude'].mean()
global_magnitude_std = acc_df['magnitude'].std()
print(f"Global magnitude stats: Mean = {global_magnitude_mean:.6f}, Std = {global_magnitude_std:.6f}")

# Compute per-activity stats for reference on cleaned data
activity_stats_df = acc_df.groupby('activity')['magnitude'].agg(['mean', 'std', 'count']).reset_index()
activity_stats_df.columns = ['activity', 'actual_mean', 'actual_std', 'sample_count']

# Ensure sufficient samples and smooth std
activity_stats_df = activity_stats_df[activity_stats_df['sample_count'] >= 100]
global_activity_std = activity_stats_df['actual_std'].mean()
activity_stats_df['actual_std'] = activity_stats_df['actual_std'].apply(lambda x: max(x, global_activity_std * 0.1))

# Debug: Print per-activity stats
print("Per-activity magnitude stats:\n", activity_stats_df[['activity', 'actual_mean', 'actual_std', 'sample_count']])

# Merge with participant characteristics for clustering
merged_df = pd.merge(acc_df, participants_df, on='participant', how='inner')

if merged_df.empty:
    print("Error: No matching participants found after merging.")
    exit(1)

# Subsample the data to reduce memory usage
subsample_fraction = 0.7  # Use 70% of the data
subsampled_df = merged_df.sample(frac=subsample_fraction, random_state=42)
print(f"Subsampled data size: {len(subsampled_df)}")

# Split into training and test sets
train_df, test_df = train_test_split(subsampled_df, test_size=0.2, random_state=42)
print(f"Training set size: {len(train_df)}, Test set size: {len(test_df)}")

# Prepare data for clustering on training set, including magnitude
X_cluster_train = train_df[['age', 'gender', 'height', 'weight', 'magnitude', 'activity']]
scaler = RobustScaler()
X_numeric_train = scaler.fit_transform(X_cluster_train[['age', 'gender', 'height', 'weight', 'magnitude']])
X_activity_train = pd.get_dummies(X_cluster_train['activity'], prefix='activity')
X_scaled_train = np.hstack([X_numeric_train, X_activity_train])

# Prepare data for test set
X_cluster_test = test_df[['age', 'gender', 'height', 'weight', 'magnitude', 'activity']]
X_numeric_test = scaler.transform(X_cluster_test[['age', 'gender', 'height', 'weight', 'magnitude']])
X_activity_test = pd.get_dummies(X_cluster_test['activity'], prefix='activity')
X_activity_test = X_activity_test.reindex(columns=X_activity_train.columns, fill_value=0)
X_scaled_test = np.hstack([X_numeric_test, X_activity_test])

# Generate k-distance plot to determine initial eps
min_samples = 50  # Default min_samples
neighbors = NearestNeighbors(n_neighbors=min_samples).fit(X_scaled_train)
distances, indices = neighbors.kneighbors(X_scaled_train)
k_distances = np.sort(distances[:, min_samples - 1])
plt.figure(figsize=(10, 6))
plt.plot(k_distances)
plt.title('K-Distance Plot (Distance to {}th Nearest Neighbor)'.format(min_samples))
plt.xlabel('Points Sorted by Distance')
plt.ylabel('Distance')
plt.grid(True)
plt.savefig('/content/drive/My Drive/ColabNotebooks/k_distance_plot.png')
plt.close()

# Suggest eps based on the knee point (e.g., 95th percentile of k-distances)
suggested_eps = np.percentile(k_distances, 95)
print(f"Global magnitude stats: Mean = {global_magnitude_mean:.6f}, Std = {global_magnitude_std:.6f}")
print(f"Suggested eps based on 95th percentile of k-distances: {suggested_eps:.3f}")

# Parameter tuning using silhouette score
eps_values = np.arange(0.1, 0.5, 0.05)  # Narrower range around suggested eps and prior best
min_samples_values = [5, 10, 15, 20]  # Lower range to capture smaller noise clusters
best_score = -1
best_params = {'eps': 0, 'min_samples': 0}

for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(X_scaled_train)
        if len(np.unique(labels)) > 1 and -1 in labels:
            score = silhouette_score(X_scaled_train, labels, sample_size=5000)
            if score > best_score:
                best_score = score
                best_params = {'eps': eps, 'min_samples': min_samples}

print(f"Per-activity magnitude stats:\n", activity_stats_df[['activity', 'actual_mean', 'actual_std', 'sample_count']])
print(f"Best parameters: eps={best_params['eps']:.1f}, min_samples={best_params['min_samples']}, Best Silhouette Score={best_score:.3f}")
print(f"Per-activity magnitude stats:\n", activity_stats_df[['activity', 'actual_mean', 'actual_std', 'sample_count']])
print(f"Best parameters: eps={best_params['eps']:.1f}, min_samples={best_params['min_samples']}, Best Silhouette Score={best_score:.3f}")

# Use best parameters for clustering
dbscan = DBSCAN(eps=best_params['eps'], min_samples=best_params['min_samples'])
train_cluster_labels = dbscan.fit_predict(X_scaled_train)
test_cluster_labels = dbscan.fit_predict(X_scaled_test)

# Add cluster labels to train and test dataframes
train_df['cluster'] = train_cluster_labels
test_df['cluster'] = test_cluster_labels

# Combine train and test dataframes
acc_df = pd.concat([train_df, test_df], axis=0)

# Assign cluster mean and std for Z-score calculation using DBSCAN clusters
cluster_means = train_df[train_df['cluster'] != -1].groupby('cluster')['magnitude'].mean().to_dict()
cluster_stds = train_df[train_df['cluster'] != -1].groupby('cluster')['magnitude'].std().to_dict()
acc_df['cluster_mu'] = acc_df['cluster'].map(cluster_means)
acc_df['cluster_sigma'] = acc_df['cluster'].map(cluster_stds).fillna(global_activity_std)
acc_df.loc[acc_df['cluster'] == -1, 'cluster_mu'] = global_magnitude_mean
acc_df.loc[acc_df['cluster'] == -1, 'cluster_sigma'] = global_magnitude_std

# Compute Z-scores using cluster-based statistics
acc_df['Z_score'] = (acc_df['magnitude'] - acc_df['cluster_mu']) / acc_df['cluster_sigma'].replace(0, np.nan)
acc_df['traditional_Z_score'] = (acc_df['magnitude'] - global_magnitude_mean) / global_magnitude_std

# Add global mean and std to output
acc_df['global_magnitude_mean'] = global_magnitude_mean
acc_df['global_magnitude_std'] = global_magnitude_std

# Apply activity-specific plausible ranges
acc_df['activity'] = acc_df['activity'].str.lower()
default_min_plausible = 0
default_max_plausible = 10
acc_df['min_plausible'] = acc_df['activity'].map(lambda x: activity_ranges.get(x, {'min': default_min_plausible})['min'])
acc_df['max_plausible'] = acc_df['activity'].map(lambda x: activity_ranges.get(x, {'max': default_max_plausible})['max'])

# Debug: Print a sample of plausible ranges applied
print("Sample of plausible ranges applied:")
print(acc_df[['activity', 'min_plausible', 'max_plausible']].drop_duplicates())

# Determine if observations are outside plausible range
acc_df['outside_plausible'] = (acc_df['magnitude'] < acc_df['min_plausible']) | (acc_df['magnitude'] > acc_df['max_plausible'])

# Determine anomaly
threshold = 2
acc_df['is_anomaly'] = np.abs(acc_df['Z_score']) > threshold
acc_df['is_anomaly_traditional'] = np.abs(acc_df['traditional_Z_score']) > threshold

# Label noise
acc_df['is_noise'] = acc_df['is_anomaly'] & acc_df['outside_plausible']
acc_df['is_noise_traditional'] = acc_df['is_anomaly_traditional'] & acc_df['outside_plausible']

# Round numerical columns in acc_df to 2 decimal places
numerical_cols = acc_df.select_dtypes(include=['float64', 'int64']).columns
acc_df[numerical_cols] = acc_df[numerical_cols].round(2)

# Save all results to first CSV
all_results_path = '/content/drive/My Drive/ColabNotebooks/all_results_clustered_optimized_dbscan_gn.csv'
try:
    acc_df.to_csv(all_results_path, index=False)
    print(f"All results saved to '{all_results_path}'.")
except Exception as e:
    print(f"Error saving all results: {e}")

# Function to introduce Generalized Normal noise
def introduce_noise(df, noise_level=0.1, noise_fraction=0.05):
    noisy_df = df.copy()
    mask = np.random.rand(len(noisy_df)) < noise_fraction
    noise = gennorm.rvs(beta=1.5, loc=0, scale=noise_level, size=len(noisy_df))
    noisy_df['magnitude'] = noisy_df['magnitude'] + noise * mask
    noisy_df['is_true_noise'] = mask.astype(int)  # Ground truth: 1 for noisy, 0 for normal
    return noisy_df

# List of noise fractions to test
noise_fractions = [0.02, 0.05, 0.10, 0.20]
results_list = []

# Iterate over noise levels
for noise_fraction in noise_fractions:
    # Introduce noise to the dataset
    noisy_df = introduce_noise(acc_df, noise_level=1.5 * global_magnitude_std, noise_fraction=noise_fraction)

    # Apply anomaly detection - Hybrid method
    noisy_df['Z_score'] = (noisy_df['magnitude'] - noisy_df['cluster_mu']) / noisy_df['cluster_sigma'].replace(0, np.nan)
    noisy_df['is_anomaly'] = np.abs(noisy_df['Z_score']) > threshold
    noisy_df['outside_plausible'] = (noisy_df['magnitude'] < noisy_df['min_plausible']) | (noisy_df['magnitude'] > noisy_df['max_plausible'])
    noisy_df['is_noise'] = noisy_df['is_anomaly'] & noisy_df['outside_plausible']

    # Apply anomaly detection - Traditional method
    noisy_df['traditional_Z_score'] = (noisy_df['magnitude'] - global_magnitude_mean) / global_magnitude_std
    noisy_df['is_anomaly_traditional'] = np.abs(noisy_df['traditional_Z_score']) > threshold
    noisy_df['is_noise_traditional'] = noisy_df['is_anomaly_traditional'] & noisy_df['outside_plausible']

    # Evaluate performance
    y_true = noisy_df['is_true_noise']
    y_pred_hybrid = noisy_df['is_noise'].astype(int)
    y_pred_traditional = noisy_df['is_noise_traditional'].astype(int)

    precision_hybrid = precision_score(y_true, y_pred_hybrid)
    recall_hybrid = recall_score(y_true, y_pred_hybrid)
    f1_hybrid = f1_score(y_true, y_pred_hybrid)

    precision_traditional = precision_score(y_true, y_pred_traditional)
    recall_traditional = recall_score(y_true, y_pred_traditional)
    f1_traditional = f1_score(y_true, y_pred_traditional)

    # Calculate percentage of noise detected
    total_noise_points = y_true.sum()
    hybrid_detected = y_pred_hybrid[y_true == 1].sum()
    traditional_detected = y_pred_traditional[y_true == 1].sum()
    hybrid_noise_percentage = (hybrid_detected / total_noise_points * 100) if total_noise_points > 0 else 0
    traditional_noise_percentage = (traditional_detected / total_noise_points * 100) if total_noise_points > 0 else 0

    # Store results for this noise level
    results_list.append({
        'Algorithm': 'DBSCAN',
        'Noise Fraction (%)': noise_fraction * 100,
        'Method': 'Hybrid (Cluster-based)',
        'Precision': precision_hybrid,
        'Recall': recall_hybrid,
        'F1 Score': f1_hybrid,
        'Noise Detected (%)': hybrid_noise_percentage
    })
    results_list.append({
        'Algorithm': 'DBSCAN',
        'Noise Fraction (%)': noise_fraction * 100,
        'Method': 'Traditional (Global)',
        'Precision': precision_traditional,
        'Recall': recall_traditional,
        'F1 Score': f1_traditional,
        'Noise Detected (%)': traditional_noise_percentage
    })

# Create consolidated results table
results_df = pd.DataFrame(results_list)
print("\nConsolidated Evaluation Results:")
print(results_df)

# Save noisy dataset and results
try:
    noisy_df.to_csv('/content/drive/My Drive/ColabNotebooks/noisy_results_dbscan_gn.csv', index=False)
    print(f"Noisy dataset saved to '/content/drive/My Drive/ColabNotebooks/noisy_results_dbscan_gn.csv'.")
    results_df.to_csv('/content/drive/My Drive/ColabNotebooks/evaluation_results_dbscan_gn.csv', index=False)
    print(f"Evaluation results saved to '/content/drive/My Drive/ColabNotebooks/evaluation_results_dbscan_gn.csv'.")
except Exception as e:
    print(f"Error saving files: {e}")

# Debug: Print summary for the last noise level
print("\nZ-score stats (noisy dataset):\n", noisy_df[['Z_score', 'traditional_Z_score']].describe())
print("Noise detection summary (hybrid, noisy dataset):\n", noisy_df['is_noise'].value_counts())
print("Noise detection summary (traditional, noisy dataset):\n", noisy_df['is_noise_traditional'].value_counts())
print(f"Total noise points introduced: {total_noise_points}")
print(f"Hybrid method detected: {hybrid_detected} ({hybrid_noise_percentage:.2f}%)")
print(f"Traditional method detected: {traditional_detected} ({traditional_noise_percentage:.2f}%)")

                        timestamp                        sensor_id  \
0        2017-10-01T12:00:00.000Z       P001_Trial_1_dws_Gyroscope   
1        2017-10-01T12:00:00.000Z       P001_Trial_1_dws_Gyroscope   
2        2017-10-01T12:00:00.000Z       P001_Trial_1_dws_Gyroscope   
3        2017-10-01T12:00:00.000Z   P001_Trial_1_dws_Accelerometer   
4        2017-10-01T12:00:00.000Z   P001_Trial_1_dws_Accelerometer   
...                           ...                              ...   
4605955  2017-10-01T16:15:53.180Z      P024_Trial_16_jog_Gyroscope   
4605956  2017-10-01T16:15:53.180Z      P024_Trial_16_jog_Gyroscope   
4605957  2017-10-01T16:15:53.180Z  P024_Trial_16_jog_Accelerometer   
4605958  2017-10-01T16:15:53.180Z  P024_Trial_16_jog_Accelerometer   
4605959  2017-10-01T16:15:53.180Z  P024_Trial_16_jog_Accelerometer   

         measurement           property  
0           1.528132       AttitudeRoll  
1          -0.733896      AttitudePitch  
2           0.696372        Attit

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import warnings
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
from scipy.stats import gennorm
from rdflib import Graph

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Paths to sensor reading csv file data and knowledge graph
data_path = "/content/drive/My Drive/ColabNotebooks/motion_sensor_readings.csv"
kg_path = "/content/drive/My Drive/ColabNotebooks/motion_sense_ssn_kg.ttl"

# Load knowledge graph
g = Graph()
try:
    g.parse(kg_path, format='turtle')
except Exception as e:
    print(f"Error parsing knowledge graph: {e}")
    exit(1)

# Query for participant characteristics
query = """
PREFIX ms: <http://example.org/motion-sense#>
SELECT ?p ?code ?age ?gender ?height ?weight
WHERE {
    ?p a ms:Participant ;
       ms:hasCode ?code ;
       ms:hasAge ?age ;
       ms:hasGender ?gender ;
       ms:hasHeightCm ?height ;
       ms:hasWeightKg ?weight .
}
"""
results = g.query(query)
participants = []
for row in results:
    participants.append({
        'participant': str(row.code).strip().upper(),
        'age': int(row.age),
        'gender': int(row.gender),
        'height': float(row.height),
        'weight': float(row.weight)
    })
participants_df = pd.DataFrame(participants)

if participants_df.empty:
    print("Error: No participants found in knowledge graph.")
    exit(1)

# Query for activity plausible ranges
query_ranges = """
PREFIX activity: <http://example.org/activity-recognition#>
SELECT ?activityCode ?min ?max
WHERE {
    ?act a activity:Activity ;
         activity:hasActivityCode ?activityCode ;
         activity:hasMinAcceleration ?min ;
         activity:hasMaxAcceleration ?max .
}
"""
results_ranges = g.query(query_ranges)
activity_ranges = {}
for row in results_ranges:
    activity_code = str(row.activityCode).strip().lower()
    activity_ranges[activity_code] = {
        'min': float(row.min),
        'max': float(row.max)
    }

if not activity_ranges:
    print("Error: No activity ranges found in knowledge graph.")
    exit(1)

# Load sensor readings csv file data
try:
    df = pd.read_csv(data_path)
    print(df)
except Exception as e:
    print(f"Error loading sensor data: {e}")
    exit(1)

# Parse sensor_id with validation
def parse_sensor_id(sid):
    parts = sid.split('_')
    if len(parts) >= 5:
        return parts[0].strip().upper(), parts[3], parts[4]
    return None, None, None

df['participant'], df['activity'], df['sensor_type'] = zip(*df['sensor_id'].apply(parse_sensor_id))
df = df.dropna(subset=['participant'])

# Filter for accelerometer data
df = df[(df['sensor_type'] == 'Accelerometer') &
        (df['property'].isin(['UserAccelerationX', 'UserAccelerationY', 'UserAccelerationZ']))]

# Convert measurement to numeric, drop invalid entries
df['measurement'] = pd.to_numeric(df['measurement'], errors='coerce')
df = df.dropna(subset=['measurement'])

if df.empty:
    print("Error: No valid accelerometer data after cleaning.")
    exit(1)

# Pivot to get X, Y, Z
acc_df = df.pivot_table(index=['timestamp', 'sensor_id', 'participant', 'activity'],
                        columns='property', values='measurement', aggfunc='first').reset_index()

# Drop rows with missing or invalid acceleration components
acc_df = acc_df.dropna(subset=['UserAccelerationX', 'UserAccelerationY', 'UserAccelerationZ'])
for col in ['UserAccelerationX', 'UserAccelerationY', 'UserAccelerationZ']:
    acc_df[col] = pd.to_numeric(acc_df[col], errors='coerce')
acc_df = acc_df.dropna(subset=['UserAccelerationX', 'UserAccelerationY', 'UserAccelerationZ'])

if acc_df.empty:
    print("Error: No valid data after pivot and cleaning.")
    exit(1)

# Compute magnitude
acc_df['magnitude'] = np.sqrt(acc_df['UserAccelerationX']**2 +
                              acc_df['UserAccelerationY']**2 +
                              acc_df['UserAccelerationZ']**2)

# Compute global mean and std for magnitude (for traditional Z-score)
global_magnitude_mean = acc_df['magnitude'].mean()
global_magnitude_std = acc_df['magnitude'].std()
print(f"Global magnitude stats: Mean = {global_magnitude_mean:.6f}, Std = {global_magnitude_std:.6f}")

# Compute per-activity stats for reference on cleaned data
activity_stats_df = acc_df.groupby('activity')['magnitude'].agg(['mean', 'std', 'count']).reset_index()
activity_stats_df.columns = ['activity', 'actual_mean', 'actual_std', 'sample_count']

# Ensure sufficient samples and smooth std
activity_stats_df = activity_stats_df[activity_stats_df['sample_count'] >= 100]
global_activity_std = activity_stats_df['actual_std'].mean()
activity_stats_df['actual_std'] = activity_stats_df['actual_std'].apply(lambda x: max(x, global_activity_std * 0.1))

# Debug: Print per-activity stats
print("Per-activity magnitude stats:\n", activity_stats_df[['activity', 'actual_mean', 'actual_std', 'sample_count']])

# Merge with participant characteristics for clustering
merged_df = pd.merge(acc_df, participants_df, on='participant', how='inner')

if merged_df.empty:
    print("Error: No matching participants found after merging.")
    exit(1)

# Use full dataset (no subsampling)
subsampled_df = merged_df  # Full dataset
print(f"Full dataset size: {len(subsampled_df)}")

# Split into training and test sets
train_df, test_df = train_test_split(subsampled_df, test_size=0.2, random_state=42)
print(f"Training set size: {len(train_df)}, Test set size: {len(test_df)}")

# Prepare data for clustering on training set, including magnitude
X_cluster_train = train_df[['age', 'gender', 'height', 'weight', 'magnitude', 'activity']]
scaler = RobustScaler()
X_numeric_train = scaler.fit_transform(X_cluster_train[['age', 'gender', 'height', 'weight', 'magnitude']])
X_activity_train = pd.get_dummies(X_cluster_train['activity'], prefix='activity')
X_scaled_train = np.hstack([X_numeric_train, X_activity_train])

# Prepare data for test set
X_cluster_test = test_df[['age', 'gender', 'height', 'weight', 'magnitude', 'activity']]
X_numeric_test = scaler.transform(X_cluster_test[['age', 'gender', 'height', 'weight', 'magnitude']])
X_activity_test = pd.get_dummies(X_cluster_test['activity'], prefix='activity')
X_activity_test = X_activity_test.reindex(columns=X_activity_train.columns, fill_value=0)
X_scaled_test = np.hstack([X_numeric_test, X_activity_test])

# Generate k-distance plot to determine initial eps
min_samples = 50  # Default min_samples
neighbors = NearestNeighbors(n_neighbors=min_samples).fit(X_scaled_train)
distances, indices = neighbors.kneighbors(X_scaled_train)
k_distances = np.sort(distances[:, min_samples - 1])
plt.figure(figsize=(10, 6))
plt.plot(k_distances)
plt.title('K-Distance Plot (Distance to {}th Nearest Neighbor)'.format(min_samples))
plt.xlabel('Points Sorted by Distance')
plt.ylabel('Distance')
plt.grid(True)
plt.savefig('/content/drive/My Drive/ColabNotebooks/k_distance_plot.png')
plt.close()

# Suggest eps based on the knee point (e.g., 95th percentile of k-distances)
suggested_eps = np.percentile(k_distances, 95)
print(f"Global magnitude stats: Mean = {global_magnitude_mean:.6f}, Std = {global_magnitude_std:.6f}")
print(f"Suggested eps based on 95th percentile of k-distances: {suggested_eps:.3f}")

# Parameter tuning using silhouette score
eps_values = np.arange(0.05, 0.2, 0.01)  # Tighter range around suggested eps
min_samples_values = [3, 5, 10, 15]  # Lower range for smaller clusters
best_score = -1
best_params = {'eps': 0, 'min_samples': 0}

for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(X_scaled_train)
        if len(np.unique(labels)) > 1 and -1 in labels:
            score = silhouette_score(X_scaled_train, labels, sample_size=5000)
            if score > best_score:
                best_score = score
                best_params = {'eps': eps, 'min_samples': min_samples}

print(f"Per-activity magnitude stats:\n", activity_stats_df[['activity', 'actual_mean', 'actual_std', 'sample_count']])
print(f"Best parameters: eps={best_params['eps']:.2f}, min_samples={best_params['min_samples']}, Best Silhouette Score={best_score:.3f}")
print(f"Per-activity magnitude stats:\n", activity_stats_df[['activity', 'actual_mean', 'actual_std', 'sample_count']])
print(f"Best parameters: eps={best_params['eps']:.2f}, min_samples={best_params['min_samples']}, Best Silhouette Score={best_score:.3f}")

# Use best parameters for clustering
dbscan = DBSCAN(eps=best_params['eps'], min_samples=best_params['min_samples'])
train_cluster_labels = dbscan.fit_predict(X_scaled_train)
test_cluster_labels = dbscan.fit_predict(X_scaled_test)

# Add cluster labels to train and test dataframes
train_df['cluster'] = train_cluster_labels
test_df['cluster'] = test_cluster_labels

# Combine train and test dataframes
acc_df = pd.concat([train_df, test_df], axis=0)

# Assign cluster mean and std for Z-score calculation using DBSCAN clusters
cluster_means = train_df[train_df['cluster'] != -1].groupby('cluster')['magnitude'].mean().to_dict()
cluster_stds = train_df[train_df['cluster'] != -1].groupby('cluster')['magnitude'].std().to_dict()
acc_df['cluster_mu'] = acc_df['cluster'].map(cluster_means)
acc_df['cluster_sigma'] = acc_df['cluster'].map(cluster_stds).fillna(global_activity_std)
acc_df.loc[acc_df['cluster'] == -1, 'cluster_mu'] = global_magnitude_mean
acc_df.loc[acc_df['cluster'] == -1, 'cluster_sigma'] = global_magnitude_std

# Compute Z-scores using cluster-based statistics
acc_df['Z_score'] = (acc_df['magnitude'] - acc_df['cluster_mu']) / acc_df['cluster_sigma'].replace(0, np.nan)
acc_df['traditional_Z_score'] = (acc_df['magnitude'] - global_magnitude_mean) / global_magnitude_std

# Add global mean and std to output
acc_df['global_magnitude_mean'] = global_magnitude_mean
acc_df['global_magnitude_std'] = global_magnitude_std

# Apply activity-specific plausible ranges
acc_df['activity'] = acc_df['activity'].str.lower()
default_min_plausible = 0
default_max_plausible = 10
acc_df['min_plausible'] = acc_df['activity'].map(lambda x: activity_ranges.get(x, {'min': default_min_plausible})['min'])
acc_df['max_plausible'] = acc_df['activity'].map(lambda x: activity_ranges.get(x, {'max': default_max_plausible})['max'])

# Debug: Print a sample of plausible ranges applied
print("Sample of plausible ranges applied:")
print(acc_df[['activity', 'min_plausible', 'max_plausible']].drop_duplicates())

# Determine if observations are outside plausible range
acc_df['outside_plausible'] = (acc_df['magnitude'] < acc_df['min_plausible']) | (acc_df['magnitude'] > acc_df['max_plausible'])

# Determine anomaly
threshold = 2
acc_df['is_anomaly'] = np.abs(acc_df['Z_score']) > threshold
acc_df['is_anomaly_traditional'] = np.abs(acc_df['traditional_Z_score']) > threshold

# Label noise
acc_df['is_noise'] = acc_df['is_anomaly'] & acc_df['outside_plausible']
acc_df['is_noise_traditional'] = acc_df['is_anomaly_traditional'] & acc_df['outside_plausible']

# Round numerical columns in acc_df to 2 decimal places
numerical_cols = acc_df.select_dtypes(include=['float64', 'int64']).columns
acc_df[numerical_cols] = acc_df[numerical_cols].round(2)

# Save all results to first CSV
all_results_path = '/content/drive/My Drive/ColabNotebooks/all_results_clustered_optimized_dbscan_gn.csv'
try:
    acc_df.to_csv(all_results_path, index=False)
    print(f"All results saved to '{all_results_path}'.")
except Exception as e:
    print(f"Error saving all results: {e}")

# Function to introduce Generalized Normal noise
def introduce_noise(df, noise_level=0.1, noise_fraction=0.05):
    noisy_df = df.copy()
    mask = np.random.rand(len(noisy_df)) < noise_fraction
    noise = gennorm.rvs(beta=1.5, loc=0, scale=noise_level, size=len(noisy_df))
    noisy_df['magnitude'] = noisy_df['magnitude'] + noise * mask
    noisy_df['is_true_noise'] = mask.astype(int)  # Ground truth: 1 for noisy, 0 for normal
    return noisy_df

# List of noise fractions to test
noise_fractions = [0.02, 0.05, 0.10, 0.20]
results_list = []

# Iterate over noise levels
for noise_fraction in noise_fractions:
    # Introduce noise to the dataset
    noisy_df = introduce_noise(acc_df, noise_level=1.5 * global_magnitude_std, noise_fraction=noise_fraction)

    # Apply anomaly detection - Hybrid method
    noisy_df['Z_score'] = (noisy_df['magnitude'] - noisy_df['cluster_mu']) / noisy_df['cluster_sigma'].replace(0, np.nan)
    noisy_df['is_anomaly'] = np.abs(noisy_df['Z_score']) > threshold
    noisy_df['outside_plausible'] = (noisy_df['magnitude'] < noisy_df['min_plausible']) | (noisy_df['magnitude'] > noisy_df['max_plausible'])
    noisy_df['is_noise'] = noisy_df['is_anomaly'] & noisy_df['outside_plausible']

    # Apply anomaly detection - Traditional method
    noisy_df['traditional_Z_score'] = (noisy_df['magnitude'] - global_magnitude_mean) / global_magnitude_std
    noisy_df['is_anomaly_traditional'] = np.abs(noisy_df['traditional_Z_score']) > threshold
    noisy_df['is_noise_traditional'] = noisy_df['is_anomaly_traditional'] & noisy_df['outside_plausible']

    # Evaluate performance
    y_true = noisy_df['is_true_noise']
    y_pred_hybrid = noisy_df['is_noise'].astype(int)
    y_pred_traditional = noisy_df['is_noise_traditional'].astype(int)

    precision_hybrid = precision_score(y_true, y_pred_hybrid)
    recall_hybrid = recall_score(y_true, y_pred_hybrid)
    f1_hybrid = f1_score(y_true, y_pred_hybrid)

    precision_traditional = precision_score(y_true, y_pred_traditional)
    recall_traditional = recall_score(y_true, y_pred_traditional)
    f1_traditional = f1_score(y_true, y_pred_traditional)

    # Calculate percentage of noise detected
    total_noise_points = y_true.sum()
    hybrid_detected = y_pred_hybrid[y_true == 1].sum()
    traditional_detected = y_pred_traditional[y_true == 1].sum()
    hybrid_noise_percentage = (hybrid_detected / total_noise_points * 100) if total_noise_points > 0 else 0
    traditional_noise_percentage = (traditional_detected / total_noise_points * 100) if total_noise_points > 0 else 0

    # Store results for this noise level
    results_list.append({
        'Algorithm': 'DBSCAN',
        'Noise Fraction (%)': noise_fraction * 100,
        'Method': 'Hybrid (Cluster-based)',
        'Precision': precision_hybrid,
        'Recall': recall_hybrid,
        'F1 Score': f1_hybrid,
        'Noise Detected (%)': hybrid_noise_percentage
    })
    results_list.append({
        'Algorithm': 'DBSCAN',
        'Noise Fraction (%)': noise_fraction * 100,
        'Method': 'Traditional (Global)',
        'Precision': precision_traditional,
        'Recall': recall_traditional,
        'F1 Score': f1_traditional,
        'Noise Detected (%)': traditional_noise_percentage
    })

# Create consolidated results table
results_df = pd.DataFrame(results_list)
print("\nConsolidated Evaluation Results:")
print(results_df)

# Save noisy dataset and results
try:
    noisy_df.to_csv('/content/drive/My Drive/ColabNotebooks/noisy_results_dbscan_gn.csv', index=False)
    print(f"Noisy dataset saved to '/content/drive/My Drive/ColabNotebooks/noisy_results_dbscan_gn.csv'.")
    results_df.to_csv('/content/drive/My Drive/ColabNotebooks/evaluation_results_dbscan_gn.csv', index=False)
    print(f"Evaluation results saved to '/content/drive/My Drive/ColabNotebooks/evaluation_results_dbscan_gn.csv'.")
except Exception as e:
    print(f"Error saving files: {e}")

# Debug: Print summary for the last noise level
print("\nZ-score stats (noisy dataset):\n", noisy_df[['Z_score', 'traditional_Z_score']].describe())
print("Noise detection summary (hybrid, noisy dataset):\n", noisy_df['is_noise'].value_counts())
print("Noise detection summary (traditional, noisy dataset):\n", noisy_df['is_noise_traditional'].value_counts())
print(f"Total noise points introduced: {total_noise_points}")
print(f"Hybrid method detected: {hybrid_detected} ({hybrid_noise_percentage:.2f}%)")
print(f"Traditional method detected: {traditional_detected} ({traditional_noise_percentage:.2f}%)")

                        timestamp                        sensor_id  \
0        2017-10-01T12:00:00.000Z       P001_Trial_1_dws_Gyroscope   
1        2017-10-01T12:00:00.000Z       P001_Trial_1_dws_Gyroscope   
2        2017-10-01T12:00:00.000Z       P001_Trial_1_dws_Gyroscope   
3        2017-10-01T12:00:00.000Z   P001_Trial_1_dws_Accelerometer   
4        2017-10-01T12:00:00.000Z   P001_Trial_1_dws_Accelerometer   
...                           ...                              ...   
4605955  2017-10-01T16:15:53.180Z      P024_Trial_16_jog_Gyroscope   
4605956  2017-10-01T16:15:53.180Z      P024_Trial_16_jog_Gyroscope   
4605957  2017-10-01T16:15:53.180Z  P024_Trial_16_jog_Accelerometer   
4605958  2017-10-01T16:15:53.180Z  P024_Trial_16_jog_Accelerometer   
4605959  2017-10-01T16:15:53.180Z  P024_Trial_16_jog_Accelerometer   

         measurement           property  
0           1.528132       AttitudeRoll  
1          -0.733896      AttitudePitch  
2           0.696372        Attit

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import silhouette_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import warnings
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
from rdflib import Graph
from google.colab import drive
from scipy.stats import gennorm
import itertools

# Mount Google Drive
drive.mount('/content/drive')

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Paths to sensor reading csv file data and knowledge graph
data_path = "/content/drive/My Drive/ColabNotebooks/motion_sensor_readings.csv"
kg_path = "/content/drive/My Drive/ColabNotebooks/motion_sense_ssn_kg.ttl"

# Load knowledge graph
g = Graph()
try:
    g.parse(kg_path, format='turtle')
except Exception as e:
    print(f"Error parsing knowledge graph: {e}")
    exit(1)

# Query for participant characteristics
query = """
PREFIX ms: <http://example.org/motion-sense#>
SELECT ?p ?code ?age ?gender ?height ?weight
WHERE {
    ?p a ms:Participant ;
       ms:hasCode ?code ;
       ms:hasAge ?age ;
       ms:hasGender ?gender ;
       ms:hasHeightCm ?height ;
       ms:hasWeightKg ?weight .
}
"""
results = g.query(query)
participants = []
for row in results:
    participants.append({
        'participant': str(row.code).strip().upper(),
        'age': int(row.age),
        'gender': int(row.gender),
        'height': float(row.height),
        'weight': float(row.weight)
    })
participants_df = pd.DataFrame(participants)

if participants_df.empty:
    print("Error: No participants found in knowledge graph.")
    exit(1)

# Query for activity plausible ranges
query_ranges = """
PREFIX activity: <http://example.org/activity-recognition#>
SELECT ?activityCode ?min ?max
WHERE {
    ?act a activity:Activity ;
         activity:hasActivityCode ?activityCode ;
         activity:hasMinAcceleration ?min ;
         activity:hasMaxAcceleration ?max .
}
"""
results_ranges = g.query(query_ranges)
activity_ranges = {}
for row in results_ranges:
    activity_code = str(row.activityCode).strip().lower()
    activity_ranges[activity_code] = {
        'min': float(row.min),
        'max': float(row.max)
    }

if not activity_ranges:
    print("Error: No activity ranges found in knowledge graph.")
    exit(1)

# Load sensor readings csv file data
try:
    df = pd.read_csv(data_path)
except Exception as e:
    print(f"Error loading sensor data: {e}")
    exit(1)

# Parse sensor_id with validation
def parse_sensor_id(sid):
    parts = sid.split('_')
    if len(parts) >= 5:
        return parts[0].strip().upper(), parts[3], parts[4]
    return None, None, None

df['participant'], df['activity'], df['sensor_type'] = zip(*df['sensor_id'].apply(parse_sensor_id))
df = df.dropna(subset=['participant'])

# Filter for accelerometer data
df = df[(df['sensor_type'] == 'Accelerometer') &
        (df['property'].isin(['UserAccelerationX', 'UserAccelerationY', 'UserAccelerationZ']))]

# Convert measurement to numeric, drop invalid entries
df['measurement'] = pd.to_numeric(df['measurement'], errors='coerce')
df = df.dropna(subset=['measurement'])

if df.empty:
    print("Error: No valid accelerometer data after cleaning.")
    exit(1)

# Pivot to get X, Y, Z
acc_df = df.pivot_table(index=['timestamp', 'sensor_id', 'participant', 'activity'],
                        columns='property', values='measurement', aggfunc='first').reset_index()

# Drop rows with missing or invalid acceleration components
acc_df = acc_df.dropna(subset=['UserAccelerationX', 'UserAccelerationY', 'UserAccelerationZ'])
for col in ['UserAccelerationX', 'UserAccelerationY', 'UserAccelerationZ']:
    acc_df[col] = pd.to_numeric(acc_df[col], errors='coerce')
acc_df = acc_df.dropna(subset=['UserAccelerationX', 'UserAccelerationY', 'UserAccelerationZ'])

if acc_df.empty:
    print("Error: No valid data after pivot and cleaning.")
    exit(1)

# Compute magnitude
acc_df['magnitude'] = np.sqrt(acc_df['UserAccelerationX']**2 +
                              acc_df['UserAccelerationY']**2 +
                              acc_df['UserAccelerationZ']**2)

# Compute global mean and std for magnitude (for traditional Z-score)
global_magnitude_mean = acc_df['magnitude'].mean()
global_magnitude_std = acc_df['magnitude'].std()
print(f"Global magnitude stats: Mean = {global_magnitude_mean:.6f}, Std = {global_magnitude_std:.6f}")

# Compute per-activity stats for reference on cleaned data
activity_stats_df = acc_df.groupby('activity')['magnitude'].agg(['mean', 'std', 'count']).reset_index()
activity_stats_df.columns = ['activity', 'actual_mean', 'actual_std', 'sample_count']

# Ensure sufficient samples and smooth std
activity_stats_df = activity_stats_df[activity_stats_df['sample_count'] >= 100]
global_activity_std = activity_stats_df['actual_std'].mean()
activity_stats_df['actual_std'] = activity_stats_df['actual_std'].apply(lambda x: max(x, global_activity_std * 0.1))

# Debug: Print per-activity stats
print("Per-activity magnitude stats:\n", activity_stats_df[['activity', 'actual_mean', 'actual_std', 'sample_count']])

# Merge with participant characteristics for clustering
merged_df = pd.merge(acc_df, participants_df, on='participant', how='inner')

if merged_df.empty:
    print("Error: No matching participants found after merging.")
    exit(1)

# Subsample the data to reduce memory usage
subsample_fraction = 0.1  # Use 99% of the data
subsampled_df = merged_df.sample(frac=subsample_fraction, random_state=42)
print(f"Subsampled data size: {len(subsampled_df)}")

# Define feature sets to test
semantic_features = ['age', 'gender', 'height', 'weight', 'activity']
feature_combinations = []
# Include magnitude-only as baseline
feature_combinations.append(['magnitude'])
# Test magnitude with each semantic feature
for r in range(1, len(semantic_features) + 1):
    for combo in itertools.combinations(semantic_features, r):
        feature_combinations.append(['magnitude'] + list(combo))

# Function to preprocess features for clustering
def preprocess_features(df, features, activity_train=None):
    numerical_features = [f for f in features if f in ['magnitude', 'age', 'height', 'weight']]
    categorical_features = [f for f in features if f in ['gender', 'activity']]

    scaler = RobustScaler()
    X_numeric = scaler.fit_transform(df[numerical_features]) if numerical_features else np.array([])

    X_categorical = []
    if 'gender' in categorical_features:
        X_gender = df['gender'].values.reshape(-1, 1)
        X_categorical.append(X_gender)

    if 'activity' in categorical_features:
        X_activity = pd.get_dummies(df['activity'], prefix='activity')
        if activity_train is not None:
            X_activity = X_activity.reindex(columns=activity_train.columns, fill_value=0)
        else:
            activity_train = X_activity
        X_categorical.append(X_activity.values)

    if X_categorical:
        X_cat = np.hstack(X_categorical) if len(X_categorical) > 1 else X_categorical[0]
        X_scaled = np.hstack([X_numeric, X_cat]) if len(X_numeric) > 0 else X_cat
    else:
        X_scaled = X_numeric

    return X_scaled, scaler, activity_train

# Function to introduce GNN noise
def introduce_gnn_noise(df, beta=1.5, scale=0.1, noise_fraction=0.05):
    noisy_df = df.copy()
    mask = np.random.rand(len(noisy_df)) < noise_fraction
    noise = gennorm.rvs(beta=beta, loc=0, scale=scale, size=len(noisy_df))
    noisy_df['magnitude'] = noisy_df['magnitude'] + noise * mask
    noisy_df['is_true_noise'] = mask.astype(int)
    return noisy_df

# Evaluate each feature combination
results_list = []
noise_fraction = 0.2  # Test at 20% noise level
threshold = 2

for features in feature_combinations:
    print(f"\nTesting feature combination: {features}")

    # Cross-validation setup
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_f1_scores = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(subsampled_df)):
        train_df = subsampled_df.iloc[train_idx]
        val_df = subsampled_df.iloc[val_idx]

        # Preprocess training data
        X_scaled_train, scaler, activity_train = preprocess_features(train_df, features)

        # Generate k-distance plot to determine initial eps
        min_samples = 50  # Starting point for min_samples
        neighbors = NearestNeighbors(n_neighbors=min_samples).fit(X_scaled_train)
        distances, indices = neighbors.kneighbors(X_scaled_train)
        k_distances = np.sort(distances[:, min_samples - 1])
        suggested_eps = np.percentile(k_distances, 95)

        # Parameter tuning using silhouette score
        eps_values = np.linspace(suggested_eps * 0.5, suggested_eps * 1.5, 5)
        min_samples_values = [20, 30, 50, 60, 70]
        best_score = -1
        best_params = {'eps': suggested_eps, 'min_samples': min_samples}

        for eps in eps_values:
            for min_samples in min_samples_values:
                dbscan = DBSCAN(eps=eps, min_samples=min_samples)
                labels = dbscan.fit_predict(X_scaled_train)
                if len(np.unique(labels)) > 1 and -1 in labels:
                    score = silhouette_score(X_scaled_train, labels, sample_size=5000)
                    if score > best_score:
                        best_score = score
                        best_params = {'eps': eps, 'min_samples': min_samples}

        # Use best parameters for clustering
        dbscan = DBSCAN(eps=best_params['eps'], min_samples=best_params['min_samples'])
        train_cluster_labels = dbscan.fit_predict(X_scaled_train)

        # Assign clusters to validation set
        X_scaled_val, _, _ = preprocess_features(val_df, features, activity_train)
        val_cluster_labels = dbscan.fit_predict(X_scaled_val)

        train_df = train_df.copy()
        val_df = val_df.copy()
        train_df['cluster'] = train_cluster_labels
        val_df['cluster'] = val_cluster_labels

        # Combine for anomaly detection
        acc_df = pd.concat([train_df, val_df], axis=0)

        # Compute cluster stats
        cluster_means = train_df[train_df['cluster'] != -1].groupby('cluster')['magnitude'].mean().to_dict()
        cluster_stds = train_df[train_df['cluster'] != -1].groupby('cluster')['magnitude'].std().to_dict()
        acc_df['cluster_mu'] = acc_df['cluster'].map(cluster_means)
        acc_df['cluster_sigma'] = acc_df['cluster'].map(cluster_stds).fillna(global_activity_std)
        acc_df.loc[acc_df['cluster'] == -1, 'cluster_mu'] = global_magnitude_mean
        acc_df.loc[acc_df['cluster'] == -1, 'cluster_sigma'] = global_magnitude_std

        # Compute Z-scores
        acc_df['Z_score'] = (acc_df['magnitude'] - acc_df['cluster_mu']) / acc_df['cluster_sigma'].replace(0, np.nan)
        acc_df['traditional_Z_score'] = (acc_df['magnitude'] - global_magnitude_mean) / global_magnitude_std

        # Apply activity-specific plausible ranges
        acc_df['activity'] = acc_df['activity'].str.lower()
        default_min_plausible = 0
        default_max_plausible = 10
        acc_df['min_plausible'] = acc_df['activity'].map(lambda x: activity_ranges.get(x, {'min': default_min_plausible})['min'])
        acc_df['max_plausible'] = acc_df['activity'].map(lambda x: activity_ranges.get(x, {'max': default_max_plausible})['max'])

        acc_df['outside_plausible'] = (acc_df['magnitude'] < acc_df['min_plausible']) | (acc_df['magnitude'] > acc_df['max_plausible'])

        # Introduce GNN noise
        noisy_df = introduce_gnn_noise(acc_df, beta=1.5, scale=2 * global_magnitude_std, noise_fraction=noise_fraction)

        # Apply anomaly detection - Hybrid method
        noisy_df['Z_score'] = (noisy_df['magnitude'] - noisy_df['cluster_mu']) / noisy_df['cluster_sigma'].replace(0, np.nan)
        noisy_df['is_anomaly'] = np.abs(noisy_df['Z_score']) > threshold
        noisy_df['outside_plausible'] = (noisy_df['magnitude'] < noisy_df['min_plausible']) | (noisy_df['magnitude'] > noisy_df['max_plausible'])
        noisy_df['is_noise'] = noisy_df['is_anomaly'] & noisy_df['outside_plausible']

        # Evaluate performance
        y_true = noisy_df['is_true_noise']
        y_pred_hybrid = noisy_df['is_noise'].astype(int)

        precision_hybrid = precision_score(y_true, y_pred_hybrid, zero_division=0)
        recall_hybrid = recall_score(y_true, y_pred_hybrid, zero_division=0)
        f1_hybrid = f1_score(y_true, y_pred_hybrid, zero_division=0)

        total_noise_points = y_true.sum()
        hybrid_detected = y_pred_hybrid[y_true == 1].sum()
        hybrid_noise_percentage = (hybrid_detected / total_noise_points * 100) if total_noise_points > 0 else 0

        # Print results for this fold
        print(f"Fold {fold + 1} Results - Features: {features}")
        print(f"  Precision: {precision_hybrid:.6f}")
        print(f"  Recall: {recall_hybrid:.6f}")
        print(f"  F1 Score: {f1_hybrid:.6f}")
        print(f"  Noise Detected (%): {hybrid_noise_percentage:.6f}")

        cv_f1_scores.append(f1_hybrid)

    # Average F1-score across folds
    avg_f1_score = np.mean(cv_f1_scores)

    # Store results for this feature combination
    results_list.append({
        'Features': ', '.join(features),
        'Avg F1 Score': avg_f1_score,
        'Precision': precision_hybrid,  # From last fold
        'Recall': recall_hybrid,
        'Noise Detected (%)': hybrid_noise_percentage
    })

# Create results DataFrame
results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values(by='Avg F1 Score', ascending=False)
print("\nFeature Selection Results (sorted by Avg F1 Score):")
print(results_df)

# Save results
try:
    results_df.to_csv('/content/drive/My Drive/ColabNotebooks/feature_selection_results_dbscan.csv', index=False)
    print(f"Feature selection results saved to '/content/drive/My Drive/ColabNotebooks/feature_selection_results_dbscan.csv'.")
except Exception as e:
    print(f"Error saving files: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Global magnitude stats: Mean = 0.753240, Std = 0.638734
Per-activity magnitude stats:
   activity  actual_mean  actual_std  sample_count
0      dws     0.575685    0.426152        131856
1      jog     1.445474    0.905656        134231
2      ups     0.447230    0.337824        157285
3      wlk     0.691151    0.471883        344288
Subsampled data size: 76766

Testing feature combination: ['magnitude']
Fold 1 Results - Features: ['magnitude']
  Precision: 0.838053
  Recall: 0.271237
  F1 Score: 0.409831
  Noise Detected (%): 27.123677
Fold 2 Results - Features: ['magnitude']
  Precision: 0.829287
  Recall: 0.277687
  F1 Score: 0.416058
  Noise Detected (%): 27.768746
Fold 3 Results - Features: ['magnitude']
  Precision: 0.829352
  Recall: 0.271794
  F1 Score: 0.409415
  Noise Detected (%): 27.179420
Fold 4 Results - Features: ['magnitude']
  Precision: 0.8